In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os.path as osp
import struct
from datetime import datetime
from pylgmath import se3op

import matplotlib.pyplot as plt

from estimate_qc import *

In [3]:
# TODO: use smrmsg.out to get the measurement covariance for the SBET
# TODO: estimate the angular acceleration
# TODO: what measurement covariance should be used for the angular acceleration?

## Get the SBET residuals

In [4]:
root = '/workspace/raid/krb/boreas/boreas-2021-09-02-11-42/'
SBET_RATE = 200.0
C_enu_ned = np.array([[0, 1, 0],[1, 0, 0],[0, 0, -1]], dtype=np.float64)
DEG_TO_RAD = np.pi / 180
ARCSECOND_TO_DEG = 1.0 / 3600.0
changeovertime = 1627387200
# Periods during which daylight savings is in effect in Toronto
dst = [(1583650800, 1604210399), (1615705200, 1636264799), (1647154800, 1667714399), (1678604400, 1699163999), (1710054000, 1730613599)]
# Glen Shields LLA reference (first position of first sequence)
lla0 = np.array([0.7641426285531766, -1.3869452891985545, 153.67129211034626], dtype=np.float64)

In [5]:
def wrapToPi(rads):
    if rads <= -np.pi:
        rads += 2 * np.pi
    if rads > np.pi:
        rads -= 2 * np.pi
    return rads

def get_T_v_i_covariance_from_nedrph_covariance(data, smr_data):
    
    lla = np.array([data[1], data[2], data[3]], dtype=np.float64)
    T_nedref_ned = RelLLAtoNED(lla, lla0)
    
    # what we get from sbet orientation (position of pos w.r.t. ned is zero)
    # note: pos frame is applanix frame here
    heading_wander = data[9]
    alpha = data[10]  # Wander angle
    
    heading_ned = heading_wander - alpha
    roll_ned = data[7]
    pitch_ned = data[8]

    def llarph_to_T_pos_enuref(h, p, r, delta_n = 0, delta_e = 0, delta_d = 0):
        T_ned_pos = np.eye(4, dtype=np.float64)
        # applanix uses different rotation conventions from our lab.
        T_ned_pos[:3, :3] = posOrientToRot(h, p, r)
        # what we need to compute is T_nedref_pos
        T_nedref_pos = np.matmul(T_nedref_ned, T_ned_pos)
        T_nedref_pos[0, 3] += delta_n
        T_nedref_pos[1, 3] += delta_e
        T_nedref_pos[2, 3] += delta_d
        # convert to ENU
        T_enu_ned = np.eye(4, dtype=np.float64)
        T_enu_ned[:3, :3] = C_enu_ned
        T_enuref_pos = np.matmul(T_enu_ned, T_nedref_pos)
        return get_inverse_tf(T_enuref_pos)

    Tbar = T_pos_enuref = llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned)

    
    n_rms = smr_data[1]
    e_rms = smr_data[2]
    d_rms = smr_data[3]
    roll_rms = smr_data[7] * ARCSECOND_TO_DEG * DEG_TO_RAD
    pitch_rms = smr_data[8] * ARCSECOND_TO_DEG * DEG_TO_RAD
    heading_rms = smr_data[9] * ARCSECOND_TO_DEG * DEG_TO_RAD

    samples = []
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned, delta_n=n_rms))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned, delta_n=-n_rms))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned, delta_e=e_rms))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned, delta_e=-e_rms))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned, delta_d=d_rms))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, roll_ned, delta_d=-d_rms))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, wrapToPi(roll_ned + roll_rms)))
    samples.append(llarph_to_T_pos_enuref(heading_ned, pitch_ned, wrapToPi(roll_ned - roll_rms)))
    samples.append(llarph_to_T_pos_enuref(heading_ned, wrapToPi(pitch_ned + pitch_rms), roll_ned))
    samples.append(llarph_to_T_pos_enuref(heading_ned, wrapToPi(pitch_ned + pitch_rms), roll_ned))
    samples.append(llarph_to_T_pos_enuref(wrapToPi(heading_ned + heading_rms), pitch_ned, roll_ned))
    samples.append(llarph_to_T_pos_enuref(wrapToPi(heading_ned - heading_rms), pitch_ned, roll_ned))
    # Sigma_in = np.diag([n_cov, e])
    Sigma = np.zeros((6, 6))
    Tbar_inv = get_inverse_tf(Tbar)
    for T in samples:
        xi = se3op.tran2vec(T @ Tbar_inv).reshape(6, 1)
        Sigma += xi @ xi.transpose()
    Sigma *= 0.5
    # print('before: trans_norm {}'.format(npla.norm([n_rms, e_rms, d_rms])))
    # print('before: rot_norm {}'.format(npla.norm([roll_rms, pitch_rms, heading_rms])))
    # print('after: trans_norm {}'.format(npla.norm(np.sqrt(np.diag(Sigma)[:3]))))
    # print('after: rot_norm {}'.format(npla.norm(np.sqrt(np.diag(Sigma)[3:]))))
    return T_pos_enuref, Sigma

## Interface for interpolating SMR data at exactly the time that we want

In [6]:
with open(osp.join(root, 'applanix', 'smrmsg.out'), 'rb') as f:
    smr_fc = f.read()
smr_size = 10 * 8  # size of each line of smrmsg.out in bytes
smr_data_all = []
smr_data_times = []
for i in range(len(smr_fc) // smr_size):
    smr_data_all.append(struct.unpack("d" * 10, smr_fc[i * smr_size: (i + 1) * smr_size]))
    smr_data_times.append(smr_data_all[-1][0])

def get_smr_data_at_time(time, gps_times, gps_lines):
    idx = np.searchsorted(gps_times, time)
    if idx >= len(gps_times):
        idx = len(gps_times) - 1
    d = abs(gps_times[idx] - time)
    if gps_times[idx] < time and idx < len(gps_times) - 1:
        if abs(gps_times[idx + 1] - time) < d:
            idx += 1
    elif gps_times[idx] > time and idx > 0:
        if abs(gps_times[idx - 1] - time) < d:
            idx -= 1

    closest = idx
    gt_time = gps_times[closest]

    def _interpolate(lower, upper, t):
        assert(len(lower) == len(upper))
        tlow = lower[0]
        tupp = upper[0]
        assert(tlow < t and t < tupp)
        delta = tupp - tlow
        if delta == 0:
            return lower
        ratio = (t - tlow) / delta
        out = []
        for low, upp in zip(lower, upper):
            out.append(low + (upp - low) * ratio)
        out[0] = t
        return out

    line = gps_lines[closest]
    if gt_time < time:
        if closest == len(gps_lines) - 1:
            return line
        line_lower = line
        line_upper = gps_lines[closest + 1]
        return _interpolate(line_lower, line_upper, time)
    elif gt_time > time:
        if closest == 0:
            return line
        line_lower = gps_lines[closest - 1]
        line_upper = line
        return _interpolate(line_lower, line_upper, time)
    elif gt_time == time:
        return line
    

In [7]:
with open(osp.join(root, 'applanix', 'sbet.out'), 'rb') as f:
    sbet_fc = f.read()
with open(osp.join(root, 'applanix', 'ros_and_gps_time.csv')) as f:
    lines = f.readlines()
    start_time = float(lines[1].split(',')[1]) - 20
    end_time = float(lines[-1].split(',')[1]) + 20
print('start_time_gps: {}'.format(start_time))
print('end_time_gps: {}'.format(end_time))
dt = datetime.fromtimestamp(start_time)
weekday = dt.isoweekday()
if weekday == 7:
    weekday = 0
g2 = weekday * 24 * 3600 + dt.hour * 3600 + dt.minute * 60 + dt.second + dt.microsecond * 1e-6
start_week = round(start_time - g2)
gpstime0 = struct.unpack("d", sbet_fc[:8])[0]
print('gpstime0: {}'.format(gpstime0))
# get timezone offset:
# Toronto time is GMT-4 or GMT-5 depending on time of year
time_zone_offset = 5 * 3600
for period in dst:
    if period[0] < start_time and start_time < period[1]:
        time_zone_offset = 4 * 3600

time_zone_offset = 0

if 'boreas-2021-01-19-15-08' in root:
    time_zone_offset = 4 * 3600  # hardcode timezone offset for this sequence only

print('START WEEK: {} TIME ZONE OFFSET: {}'.format(start_week, time_zone_offset))
start_gps = start_time + time_zone_offset - start_week
end_gps = end_time + time_zone_offset - start_week
if start_time > changeovertime:
    start_gps += 18  # UTC --> GPS
    end_gps += 18
print('start_gps: {} end_gps: {}'.format(start_gps, end_gps))

sbet_size = 17 * 8
# smr_size = 10 * 8  # size of each line of smrmsg.out in bytes

sbet_times = []
sbet_data_all = []

skip = 20  # only use every skipth data from the SBET

poses = []
pos_covs = []
ang_vels = []
vels = []
vel_covs = []
accels = []
ang_accels = []

w_prev = np.zeros((3, 1))

for i in list(range(len(sbet_fc) // sbet_size))[::skip]:
    # SBET Data:
    data = struct.unpack("d" * 17, sbet_fc[i * sbet_size: (i + 1) * sbet_size])
    sbet_times.append(data[0])
    sbet_data_all.append(data)
    if data[0] < start_gps or data[0] > end_gps:
        continue
    # convert lat, lng to Easting, Northing in UTM
    latitude = data[1]
    longitude = data[2]
    lla = np.array([data[1], data[2], data[3]], dtype=np.float64)
    T_nedref_ned = RelLLAtoNED(lla, lla0)

    # what we get from sbet orientation (position of pos w.r.t. ned is zero)
    # note: pos frame is applanix frame here
    # heading_wander = data[9]
    alpha = data[10]  # Wander angle
    # heading_ned = heading_wander - alpha
    # roll_ned = data[7]
    # pitch_ned = data[8]

    smr_data = get_smr_data_at_time(data[0], smr_data_times, smr_data_all)
    T_pos_enuref, Sigma = get_T_v_i_covariance_from_nedrph_covariance(data, smr_data)
    T_enuref_pos = get_inverse_tf(T_pos_enuref)

    v_x = data[4]
    v_y = data[5]
    v_n = v_x * np.cos(alpha) - v_y * np.sin(alpha)   # Velocity in north direction
    v_e = -v_x * np.sin(alpha) - v_y * np.cos(alpha)  # Velocity in east direction
    
    # rotate velocity from local ned to reference ned
    v_nedref = np.matmul(T_nedref_ned[:3, :3], np.array([v_n, v_e, -data[6]], np.float64))
    v_e = v_nedref[1]
    v_n = v_nedref[0]
    v_up = -v_nedref[2]

    v = T_pos_enuref[:3, :3] @ np.array([v_e, v_n, v_up]).reshape(3, 1)
    a = np.array([data[11], data[12], data[13]]).reshape(3, 1)
    w = np.array([data[14], data[15], data[16]]).reshape(3, 1)
    wa = (w - w_prev) * SBET_RATE / skip  # finite difference approximation

    v_rms = np.array([smr_data[5], smr_data[4], smr_data[6]]).reshape(3, 1)
    v_cov = np.diag(v_rms.squeeze()**2)
    C = T_pos_enuref[:3, :3]
    v_cov = C @ v_cov @ C.T

    poses.append(T_pos_enuref)
    pos_covs.append(Sigma)
    vels.append(v.squeeze())
    ang_vels.append(w.squeeze())
    accels.append(a.squeeze())
    vel_covs.append(v_cov)
    ang_accels.append(wa.squeeze())
    
    # T_enuref_pos = get_transform([float(x) for x in gtlines[i]])

    # SMR Data:
    # smr_data = struct.unpack("d" * 8, smr_fc[i * smr_size: (i + 1) * smr_size])
    # v_n = ...
    # v_e = ...
    # v_d = ...
    # v_nedref = np.matmul(T_nedref_ned[:3, :3], np.array([v_n, v_e, -v_d], np.float64))
    # v_e = v_nedref[1]
    # v_n = v_nedref[0]
    # v_up = -v_nedref[2]
    # T_enuref_pos = ...
    # body_rate_rms = get_inverse_tf(T_enuref_pos)[:3, :3] @ np.array([v_e, v_n, v_up])
    w_prev = np.copy(w)

start_time_gps: 1630597311.036218
end_time_gps: 1630598384.6390462
gpstime0: 401729.0551340226
START WEEK: 1630195200 TIME ZONE OFFSET: 0
start_gps: 402129.03621792793 end_gps: 403202.63904619217


In [8]:
poses = np.expand_dims(np.array(poses), axis=0)
pos_covs = np.expand_dims(np.array(pos_covs), axis=0)
accels = np.expand_dims(np.array(accels), axis=0)
vels = np.expand_dims(np.array(vels), axis=0)
ang_vels = np.expand_dims(np.array(ang_vels), axis=0)
vel_covs = np.expand_dims(np.array(vel_covs), axis=0)
ang_accels = np.expand_dims(np.array(ang_accels), axis=0)

In [9]:
# print(np.diag((v_rms**2).squeeze()).shape)
print(poses.shape)
print(pos_covs.shape)
print(accels.shape)
print(vels.shape)
print(ang_vels.shape)
print(vel_covs.shape)
print(ang_accels.shape)

(1, 10736, 4, 4)
(1, 10736, 6, 6)
(1, 10736, 3)
(1, 10736, 3)
(1, 10736, 3)
(1, 10736, 3, 3)
(1, 10736, 3)


In [10]:
dim = 6
gamma1 = []
gamma2 = []
t = 0
for i in range(poses.shape[1] - 1):
    g1 = np.zeros((dim * 3, 1))
    g2 = np.zeros((dim * 3, 1))
    g1[6 : 9] = vels[t, i].reshape(3, 1)
    g1[9 : 12] = ang_vels[t, i].reshape(3, 1)
    g1[12 : 15] = accels[t, i].reshape(3, 1)
    g1[15 : 18] = ang_accels[t, i].reshape(3, 1)
    xi_21 = se3op.tran2vec(poses[t, i + 1] @ get_inverse_tf(poses[t, i]))
    g2[:6] = xi_21
    J_21_inv = se3op.vec2jacinv(xi_21)
    w2 = np.array([vels[t, i + 1], ang_vels[t, i + 1]]).reshape(6, 1)
    temp = J_21_inv @ w2
    g2[6 : 12] = temp
    dw2 = np.array([accels[t, i + 1], ang_accels[t, i + 1]]).reshape(6, 1)
    g2[12 : 18] = -0.5 * se3op.curlyhat(temp) @ w2 + J_21_inv @ dw2
    gamma1.append(g1)
    gamma2.append(g2)
gamma1 = np.expand_dims(np.array(gamma1), axis=0)
gamma2 = np.expand_dims(np.array(gamma2), axis=0)

## Train GP Prior

In [15]:
import scipy.linalg as spla
import scipy.sparse as sp_sparse
from sksparse.cholmod import cholesky

dim = 6
qc = np.ones(dim)
ad = np.ones(dim)
dt = 1.0 / (SBET_RATE / skip)
lr = 1.0

R_w = np.diag([0.04898588, 0.06860401, 0.04319808])
R_wa = (2 * R_w) * (SBET_RATE / skip)**2
R_a = np.diag([0.43000461, 0.33698219, 0.29589244])
R_aa_k = np.zeros((dim, dim))
R_aa_k[:3, :3] = R_a
R_aa_k[3:, 3:] = R_wa

params = []
cost = []
lrs = []

N = 1000

K = gamma1.shape[1]

ad_prev = ad
qc_prev = qc
cost_prev = 0

D = dim * 3
Q = sp_sparse.lil_array((K * D, K * D))
e = np.zeros((K * D, 1))
dQ_da = sp_sparse.lil_array((K * D, K * D))
dQ_dqc = sp_sparse.lil_array((K * D, K * D))
de_da = np.zeros((K * D, 1))

for it in range(N):
    params.append([qc, ad])
    
    Phi = get_tran_singer(dt, ad)
    C1 = Phi[:dim, -dim:]
    C2 = Phi[dim : 2 * dim, -dim:]
    C3 = Phi[-dim:, -dim:]
    
    Qk = get_q_singer(dt, ad, qc)
    # Qk_inv = npla.inv(Qk)
    
    jac_qc = 0
    jac_ad = 0
    cost = 0
    K = gamma1.shape[1]
    T = poses.shape[0]
    s = 0
    
    t = 0
    # for t in range(T):
    jac_qc_tmp = 0
    jac_ad_tmp = 0
    for i in range(K):
        e[i * D : (i + 1) * D] = gamma2[t, i] - Phi @ gamma1[t, i]
        
        R_vv_k = np.zeros((dim, dim))
        R_vv_k[:3, :3] = vel_covs[t, i]
        R_vv_k[3:, 3:] = R_w

        R_vv_k2 = np.zeros((dim, dim))
        R_vv_k2[:3, :3] = vel_covs[t, i + 1]
        R_vv_k2[3:, 3:] = R_w

        R_k = np.zeros((D, D))
        R_k[:dim, :dim] = pos_covs[t, i]
        R_k[dim:2*dim, dim:2*dim] = R_vv_k
        R_k[-dim:, -dim:] = R_aa_k

        R_k2 = np.zeros((D, D))
        R_k2[:dim, :dim] = pos_covs[t, i + 1]
        R_k2[dim:2*dim, dim:2*dim] = R_vv_k2
        R_k2[-dim:, -dim:] = R_aa_k
        
        # cov_kk = np.zeros((D, D))
        # cov_kk[:dim, :dim] = (
        #     pos_covs[t, i + 1] + pos_covs[t, i] + 
        #     C1 @ R_aa_k @ C1.T + dt**2 * R_vv_k
        # )
        # cov_kk[:dim, dim : 2 * dim] = dt * R_vv_k + C1 @ R_aa_k @ C2.T
        # cov_kk[dim : 2 * dim, :dim] = cov_kk[:dim, dim : 2 * dim].T
        # cov_kk[:dim, 2 * dim : 3 * dim] = C1 @ R_aa_k @ C3.T
        # cov_kk[2 * dim : 3 * dim, :dim] = cov_kk[:dim, 2 * dim : 3 * dim].T
        # cov_kk[dim : 2 * dim, dim : 2 * dim] = (
        #     R_vv_k + R_vv_k2 + C2 @ R_aa_k @ C2.T
        # )
        # cov_kk[dim : 2 * dim, 2 * dim : 3 * dim] = C2 @ R_aa_k @ C3.T
        # cov_kk[2 * dim : 3 * dim, dim : 2 * dim] = cov_kk[dim : 2 * dim, 2 * dim : 3 * dim].T
        # cov_kk[-dim:, -dim:] = C3 @ R_aa_k @ C3.T + R_aa_k

        # Q[i * D : (i + 1) * D, i * D : (i + 1) * D] = cov_kk + Qk
        Q[i * D : (i + 1) * D, i * D : (i + 1) * D] = R_k2 + Phi @ R_k @ Phi.T + Qk
        
        if i < K - 1:
            # cov_21 = np.zeros((D, D))
            # cov_21[:dim, :dim] = -pos_covs[t, i + 1]
            # cov_21[:dim, dim : 2 * dim] = -dt * R_vv_k2
            # cov_21[:dim, 2 * dim : 3 * dim] = -C1 @ R_aa_k
            # cov_21[dim : 2 * dim, dim : 2 * dim] = -R_vv_k2
            # cov_21[dim : 2 * dim, 2 * dim : 3 * dim] = -C2 @ R_aa_k
            # cov_21[-dim:, -dim:] = -C3 @ R_aa_k
            # cov_21[dim : 2 * dim, :dim] = cov_21[:dim, dim : 2 * dim].T
            # cov_21[2 * dim : 3 * dim, :dim] = cov_21[:dim, 2 * dim : 3 * dim].T
            # cov_21[2 * dim : 3 * dim, dim : 2 * dim] = cov_21[dim : 2 * dim, 2 * dim : 3 * dim].T
            cov_21 = -R_k @ Phi.T
            Q[(i + 1) * D : (i + 2) * D, i * D : (i + 1) * D] = cov_21
            Q[i * D : (i + 1) * D, (i + 1) * D : (i + 2) * D] = cov_21.T
        
        # cost += ek.T @ Qk_inv @ ek
        # if ad >= 0.05:

        # dQk_dqc = get_q_singer(dt, ad, np.ones(dim))
        # dQ_dqc[i * D : (i + 1) * D, i * D : (i + 1) * D] = dQk_dqc
        # # Note: be careful, if ad < 0.05, dQk_da for that dim is set to zero...
        # dQk_da = get_jac_Qk_alpha(dt, ad, qc)
        
        # dC_da = get_jac_C_alpha(dt, ad)
        # dC1_da = dC_da[:dim, :dim]
        # dC2_da = dC_da[dim : 2 * dim, :dim]
        # dC3_da = dC_da[-dim:, :dim]

        # dcov_kk_da = np.zeros((D, D))
        # dcov_kk_da[:dim, :dim] = 2 * C1 @ R_aa_k @ dC1_da
        # dcov_kk_da[:dim, dim : 2 * dim] = C2 @ R_aa_k @ dC1_da + C1 @ R_aa_k @ dC2_da
        # dcov_kk_da[dim : 2 * dim, :dim] = dcov_kk_da[:dim, dim : 2 * dim].T
        # dcov_kk_da[:dim, 2 * dim : 3 * dim] = C3 @ R_aa_k @ dC1_da + C1 @ R_aa_k @ dC3_da
        # dcov_kk_da[2 * dim : 3 * dim, :dim] = dcov_kk_da[:dim, 2 * dim : 3 * dim].T
        # dcov_kk_da[dim : 2 * dim, dim : 2 * dim] = 2 * C2 @ R_aa_k @ dC2_da
        # dcov_kk_da[dim : 2 * dim, 2 * dim : 3 * dim] = C3 @ R_aa_k @ dC2_da + C2 @ R_aa_k @ dC3_da
        # dcov_kk_da[2 * dim : 3 * dim, dim : 2 * dim] = dcov_kk_da[dim : 2 * dim, 2 * dim : 3 * dim].T
        # dcov_kk_da[-dim:, -dim:] = 3 * C3 @ R_aa_k @ dC3_da

        # dQ_da[i * D : (i + 1) * D, i * D : (i + 1) * D] = dcov_kk_da + dQk_da

        # de_da[i * D : (i + 1) * D] = get_jac_ek_alpha(dt, ad, gamma1[t, i].reshape(-1, 1))

        # if i < K - 1:
        #     dcov_21_da = np.zeros((D, D))
        #     dcov_21_da[:dim, 2 * dim : 3 * dim] = -R_aa_k @ dC1_da
        #     dcov_21_da[2 * dim : 3 * dim, :dim] = dcov_21_da[:dim, 2 * dim : 3 * dim].T
        #     dcov_21_da[dim : 2 * dim, 2 * dim : 3 * dim] = -R_aa_k @ dC2_da
        #     dcov_21_da[2 * dim : 3 * dim, dim : 2 * dim] = dcov_21_da[dim : 2 * dim, 2 * dim : 3 * dim].T
        #     dcov_21_da[-dim:, -dim:] = -R_aa_k @ dC3_da
        #     dQ_da[(i + 1) * D : (i + 2) * D, i * D : (i + 1) * D] = dcov_21_da
        #     dQ_da[i * D : (i + 1) * D, (i + 1) * D : (i + 2) * D] = dcov_21_da.T

    Q = 0.5 * (Q + Q.T)
    break
    factor = cholesky(Q, beta=0, mode="simplicial", ordering_method="natural")
    Q_inv_e = factor(e)
    # Q_inv_dQ_da = factor(dQ_da)
    # Q_inv_dQ_dqc = factor(dQ_dqc)

    dJ_dqc = -0.5 * Q_inv_e.T @ dQ_dqc @ Q_inv_e + 0.5 * (Q_inv_dQ_dqc).trace()
    dJ_da = -0.5 * Q_inv_e.T @ dQ_da @ Q_inv_e + Q_inv_e.T @ de_da + 0.5 * (Q_inv_dQ_da).trace()

    print(dJ_dqc)
    print(dJ_da)
    break
    
        # jac_ad_tmp += -0.5 * np.trace(sym(e @ e.T) @ Qk_inv @ dQk_da @ Qk_inv)
        # jac_ad_tmp += e.T @ Qk_inv @ get_jac_ek_alpha(dt, ad, gamma1[t, i, 0:3*dim].reshape(3, 1))
        # s += e.T @ Delta_k_inv @ e
    # jac_ad += jac_ad_tmp / K + 0.5 * np.trace(Qk_inv @ dQk_da)
    # cost += K * np.log(npla.det(Qk))
    # jac_ad /= T
    # cost /= T
    
    print("cost: {:10.4f} qc: {:10.4f} jac_ad: {:10.4f} ad: {:10.4f}".format(cost.item(), qc, jac_ad.item(), ad))
    qc = (s / (3 * K * T)).item()
    ad = ad - lr * jac_ad.item()
    if ad < 0.05:
        ad = 0.0
        jac_ad = 0
    costs.append(cost.item())
    
    if np.abs(ad - ad_prev) < 1.0e-3 and np.abs(qc - qc_prev) < 1.0e-3:
        print("Parameters converged!")
        break
    elif cost.item() - cost_prev > 0:
        print("Min cost reached")
        break
        
    ad_prev = ad
    qc_prev = qc
    cost_prev = cost.item()

KeyboardInterrupt: 

In [11]:
def logdet_blk_tridiag1(A, dim=18):
    num_blks = int(A.shape[0] / dim)
    Lambda_k = A[:dim, :dim].toarray()
    logdet = np.log(np.linalg.det(Lambda_k))
    temp = 1
    flag = 0
    for i in range(1, num_blks):
        Ak = A[i * dim : (i + 1) * dim, i * dim : (i + 1) * dim].toarray()
        Ckm1 = A[i * dim : (i + 1) * dim, (i - 1) * dim : i * dim].toarray()
        Bkm1 = A[(i - 1) * dim : i * dim, i * dim : (i + 1) * dim].toarray()
        Lambda_k = Ak - Ckm1 @ np.linalg.inv(Lambda_k) @ Bkm1

        det = np.linalg.det(Lambda_k)
        
        if det < 0:
            if flag == 0:
                temp = det
                flag = 1
            if flag == 1:
                logdet += np.log(det * temp)
                flag = 0
        else:
          logdet += np.log(det)
    # T = np.zeros((2 * dim, 2 * dim))
    # T[-dim:, :dim] = np.eye(dim)
    # i = 0
    # Ak = A[i * dim : (i + 1) * dim, i * dim : (i + 1) * dim].toarray()
    # Bk = A[i * dim : (i + 1) * dim, (i + 1) * dim : (i + 2) * dim].toarray()
    # BB = np.copy(Bk)
    # Bkinv = np.linalg.inv(Bk)
    # T[:dim, :dim] = -Bkinv @ Ak
    # T[:dim, -dim:] = -Bkinv
    
    # for i in range(1, num_blks):
    #     Ak = A[i * dim : (i + 1) * dim, i * dim : (i + 1) * dim].toarray()
    #     Ckm1 = A[i * dim : (i + 1) * dim, (i - 1) * dim : i * dim].toarray()
    #     if i < num_blks - 1:
    #         Bk = A[i * dim : (i + 1) * dim, (i + 1) * dim : (i + 2) * dim].toarray()
    #         BB = BB @ Bk
    #         Bkinv = np.linalg.inv(Bk)
    #     else:
    #         Bkinv = np.eye(dim)
    #     Tk = np.zeros((2 * dim, 2 * dim))
    #     Tk[-dim:, :dim] = np.eye(dim)
    #     Tk[:dim, :dim] = -Bkinv @ Ak
    #     Tk[:dim, -dim:] = -Bkinv @ Ckm1
    #     T = Tk @ T
    # nm = 1 # TODO
    # det = (-1)**nm * np.linalg.det(T[:dim, :dim]) * np.linalg.det(BB)
    return logdet

In [12]:
def logdet_blk_tridiag(A, dim=18):
    factor = cholesky(A.tocsc(), beta=0, mode="simplicial", ordering_method="natural")
    L = factor.LD()
    temp = 1
    flag = 0
    logdet = 0.0
    for i in range(L.shape[0]):
        if L[i,i] < 0:
            if flag == 0:
                temp = L[i,i]
                flag = 1
            if flag == 1:
                logdet += np.log(L[i,i] * temp)
                flag = 0
        else:
          logdet += np.log(L[i, i])
    assert flag == 0
    return logdet

In [130]:
# print(Q[-18:, -18:].toarray())
print(np.sum(np.linalg.eigvals(Q[:10*18*3, :10*18*3].toarray()) <= 0.0))
print(np.linalg.eigvals(Qk) > 0.0)
# print(np.linalg.eigvals(cov_kk) > 0.0)

0
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [131]:
print(Q.shape)
print(Q.shape[0] / 18)

(193230, 193230)
10735.0


In [137]:
start = time()
logdet = logdet_blk_tridiag1(Q)
print('logdet: {}'.format(logdet))
print('time: {}'.format(time() - start))

logdet: -946793.4597271177
time: 9.83336853981018


In [136]:
start = time()
logdet = logdet_blk_tridiag(Q)
print('logdet: {}'.format(logdet))
print('time: {}'.format(time() - start))

logdet: -932555.9159553277
time: 15.187644243240356


In [135]:
start = time()
factor = cholesky(Q.tocsc(), beta=0, mode="simplicial", ordering_method="natural")
Q_inv_e = factor(e)
# Q_inv_e = sp_sparse.linalg.spsolve(Q.tocsc(), e)[..., None]
cost = 0.5 * e.T @ Q_inv_e  # ** this should be positive, right?
print(cost)
print(time() - start)

[[2.44004111e+10]]
0.3103141784667969


In [88]:
print(np.max(e))
print(np.min(e))
print(np.max(Q_inv_e))
print(np.min(Q_inv_e))

9.90956174025673
-10.235130920602337
512074710.56558484
-382051672.2448657


In [73]:
print(np.sum(Q_inv_e < 0))

97621


In [51]:
from time import time
factor = cholesky(Q.tocsc(), beta=0, mode="simplicial", ordering_method="natural")
start = time()
print('det: {}'.format(factor.logdet()))
print(time() - start)

det: nan
0.008092164993286133


/tmp/ipykernel_15672/277657336.py:4: RuntimeWarning: invalid value encountered in log
  print('det: {}'.format(factor.logdet()))


In [52]:
from time import time
factor = cholesky(Q, beta=0, mode="simplicial", ordering_method="natural")
start = time()
L = factor.LD()
# L = D.tolil()
# logdet = 0.0
# for i in range(L.shape[0]):
#     logdet += np.log(L[i, i]**2)
print(time() - start)

temp = 1
flag = 0
logdet = 0.0
for i in range(L.shape[0]):
    if L[i,i] < 0:
        if flag == 0:
            temp = L[i,i]
            flag = 1
        if flag == 1:
            logdet += np.log(L[i,i] * temp)
            flag = 0
    else:
      logdet += np.log(L[i, i])

print(logdet)

/tmp/ipykernel_16354/1371997529.py:2: CholmodTypeConversionWarning: converting matrix of class lil_array to CSC format
  factor = cholesky(Q, beta=0, mode="simplicial", ordering_method="natural")


0.02532196044921875
-933046.289308364


In [50]:
L, D = factor.L_D()
print(L[:3, :3].toarray())
print(D.tolil()[:3, :3].toarray())

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.58718124e-04  1.00000000e+00  0.00000000e+00]
 [-1.91214961e-02  3.76718716e-02  1.00000000e+00]]
[[0.00038702 0.         0.        ]
 [0.         0.00038527 0.        ]
 [0.         0.         0.00085435]]


In [51]:
LD = factor.LD()
print(LD[:3, :3].toarray())

[[ 0.00038702  0.          0.        ]
 [-0.00065872  0.00038527  0.        ]
 [-0.0191215   0.03767187  0.00085435]]


In [13]:
B = sp_sparse.linalg.splu(Q.tocsc(), permc_spec='NATURAL', options=dict(Equil=False, IterRefine='SINGLE'))


In [32]:
print(B.L.shape)
# print(np.diag(B.U[:10,:10].toarray()))
# count = 0
temp = 1
flag = 0
logdet = 0.0
# for i in range(B.U.shape[0]):
#     if B.U[i,i] < 0:
#         if flag == 0:
#             temp = B.U[i,i]
#             flag = 1
#         if flag == 1:
#             logdet += np.log(B.U[i,i] * temp)
#             flag = 0
#     else:
#       logdet += np.log(B.U[i, i])
# # print(count)
# print(logdet)
# logdet = 1.0
for i in range(B.U.shape[0]):
    logdet += np.log(B.U[i, i]**2)
print(logdet)
# print(B.U[-3,-3])

(193230, 193230)
-1863027.6343694986


## Train Singer (Numerical)

In [13]:
def singer_error(params, dt, dim, R_w, R_aa_k, gamma1, gamma2):
    print(params)
    ad = params[:dim]
    qc = params[-dim:]
    D = dim * 3
    K = gamma1.shape[1]
    Q = sp_sparse.lil_array((K * D, K * D))
    e = np.zeros((K * D, 1))

    Phi = get_tran_singer(dt, ad)
    C1 = Phi[:dim, -dim:]
    C2 = Phi[dim : 2 * dim, -dim:]
    C3 = Phi[-dim:, -dim:]
    
    Qk = get_q_singer(dt, ad, qc)

    # T = gamma1.shape[0]
    # s = 0
    
    t = 0
    # for t in range(T):
    for i in range(K):
        e[i * D : (i + 1) * D] = gamma2[t, i] - Phi @ gamma1[t, i]
        
        R_vv_k = np.zeros((dim, dim))
        R_vv_k[:3, :3] = vel_covs[t, i]
        R_vv_k[3:, 3:] = R_w

        R_vv_k2 = np.zeros((dim, dim))
        R_vv_k2[:3, :3] = vel_covs[t, i + 1]
        R_vv_k2[3:, 3:] = R_w

        R_k = np.zeros((D, D))
        R_k[:dim, :dim] = pos_covs[t, i]
        R_k[dim:2*dim, dim:2*dim] = R_vv_k
        R_k[-dim:, -dim:] = R_aa_k

        R_k2 = np.zeros((D, D))
        R_k2[:dim, :dim] = pos_covs[t, i + 1]
        R_k2[dim:2*dim, dim:2*dim] = R_vv_k2
        R_k2[-dim:, -dim:] = R_aa_k
        
        Q[i * D : (i + 1) * D, i * D : (i + 1) * D] = R_k2 + Phi @ R_k @ Phi.T + Qk
        
        if i < K - 1:
            cov_21 = -R_k @ Phi.T
            Q[(i + 1) * D : (i + 2) * D, i * D : (i + 1) * D] = cov_21
            Q[i * D : (i + 1) * D, (i + 1) * D : (i + 2) * D] = cov_21.T
    Q = 0.5 * (Q + Q.T)
    factor = cholesky(Q.tocsc(), beta=0, mode="simplicial", ordering_method="natural")
    Q_inv_e = factor(e)
    cost = 0.5 * e.T @ Q_inv_e + 0.5 * logdet_blk_tridiag1(Q)
    print(cost.item())
    return cost.item()

In [ ]:
from scipy.optimize import minimize

x0 = np.array([ 3.85528515, 3.84949172, 3.88049649, 4.01160905, 4.01343317, 3.84995573,  6.18108869e+04, 7.71192323e+07, 3.45732462e+03, 7.34543263e+02,
 7.13707405e+02, 9.90013180e+03])
res = minimize(
    singer_error,
    x0,
    method='nelder-mead',
    args=(dt, dim, R_w, R_aa_k, gamma1, gamma2),
    options={'xatol': 1e-8, 'disp': True},
    bounds=[(0.0, 10.0),(0.0, 10.0),(0.0, 10.0),(0.0, 10.0),(0.0, 10.0),(0.0, 10.0),(1.0e-10, 1.0e8),(1.0e-10, 1.0e8),(1.0e-10, 1.0e8),(1.0e-10, 1.0e8),(1.0e-10, 1.0e8),(1.0e-10, 1.0e8)]
)
print(res)

[3.85528515e+00 3.84949172e+00 3.88049649e+00 4.01160905e+00
 4.01343317e+00 3.84995573e+00 6.18108869e+04 7.71192323e+07
 3.45732462e+03 7.34543263e+02 7.13707405e+02 9.90013180e+03]
110851.2941281118
[4.04804941e+00 3.84949172e+00 3.88049649e+00 4.01160905e+00
 4.01343317e+00 3.84995573e+00 6.18108869e+04 7.71192323e+07
 3.45732462e+03 7.34543263e+02 7.13707405e+02 9.90013180e+03]
110757.78127859016
[3.85528515e+00 4.04196631e+00 3.88049649e+00 4.01160905e+00
 4.01343317e+00 3.84995573e+00 6.18108869e+04 7.71192323e+07
 3.45732462e+03 7.34543263e+02 7.13707405e+02 9.90013180e+03]
110757.27674308918
[3.85528515e+00 3.84949172e+00 4.07452131e+00 4.01160905e+00
 4.01343317e+00 3.84995573e+00 6.18108869e+04 7.71192323e+07
 3.45732462e+03 7.34543263e+02 7.13707405e+02 9.90013180e+03]
110756.70018584759
[3.85528515e+00 3.84949172e+00 3.88049649e+00 4.21218950e+00
 4.01343317e+00 3.84995573e+00 6.18108869e+04 7.71192323e+07
 3.45732462e+03 7.34543263e+02 7.13707405e+02 9.90013180e+03]
11075

In [162]:
print(res)

       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: -1.3668659536916794e+16
             x: [ 2.850e-04  1.290e-04  1.995e-04  6.260e+00  6.918e+00
                  5.198e+00  7.001e+04  8.438e+07  2.636e+03  2.643e+01
                  4.161e+02  9.696e+00]
           nit: 682
          nfev: 2400
 final_simplex: (array([[ 2.850e-04,  1.290e-04, ...,  4.161e+02,
                         9.696e+00],
                       [ 2.850e-04,  1.290e-04, ...,  4.161e+02,
                         9.696e+00],
                       ...,
                       [ 2.850e-04,  1.290e-04, ...,  4.161e+02,
                         9.696e+00],
                       [ 2.850e-04,  1.290e-04, ...,  4.161e+02,
                         9.696e+00]]), array([-1.367e+16, -1.367e+16, ..., -1.367e+16, -1.367e+16]))


In [27]:
from time import time
start = time()
# L = sp_sparse.linalg.factorized(Q)
# XX = L(dQ_da)
# XX = sp_sparse.linalg.spsolve(Q.tocsc(), dQ_da.tocsc(), permc_spec='NATURAL')
print(time() - start)

lu = sp_sparse.linalg.splu(Q, permc_spec='NATURAL', options=dict(Equil=False, IterRefine='SINGLE'))
start = time()
x = lu.solve(e)
print(time() - start)

factor = cholesky(Q.tocsc(), beta=0, mode="simplicial", ordering_method="natural")
start = time()
Q_inv_e = factor(e)
print(time() - start)

3.457069396972656e-05
0.033678531646728516
0.012048959732055664


In [15]:
print(Q.shape)

(193230, 193230)


In [60]:
factor = cholesky(Q, beta=0, mode="simplicial", ordering_method="natural")
Q_inv_e = factor(e)
Q_inv_dQ_da = factor(dQ_da)
Q_inv_dQ_dqc = factor(dQ_dqc)

dJ_dqc = -0.5 * Q_inv_e.T @ dQ_dqc @ Q_inv_e + 0.5 * (Q_inv_dQ_dqc).trace()
dJ_da = -0.5 * Q_inv_e.T @ dQ_da @ Q_inv_e + Q_inv_e.T @ de_da + 0.5 * (Q_inv_dQ_da).trace()

print(dJ_dqc)
print(dJ_da)

/tmp/ipykernel_284/2975500000.py:1: CholmodTypeConversionWarning: converting matrix of class lil_array to CSC format
  factor = cholesky(Q, beta=0, mode="simplicial", ordering_method="natural")
/tmp/ipykernel_284/2975500000.py:3: CholmodTypeConversionWarning: converting matrix of class lil_array to CSC format
  Q_inv_dQ_da = factor(dQ_da)


CholmodTooLargeError: ../Core/cholmod_memory.c:335: problem too large (code -3)

## Train Singer (No Measurement Noise)

Note: solve for the correct Qc first, then do gradient descent to get the correct alpha

In [143]:
def sym(A: np.ndarray) -> np.ndarray:
    return A + A.T - A * np.eye(A.shape[0])

In [171]:
params = np.array([1.60452473e-03, 3.65553342e-04, 1.00625552e-03, 6.08698541e+00,
 6.59485984e+00, 6.11953882e-04, 6.97434629e+04, 8.29918198e+07,
 2.49382838e+03, 2.64178291e+01, 5.01105621e+02, 3.94161955e-01])
ad = params[:dim]
qc = params[-dim:]

qc = np.ones(dim)
ad = np.ones(dim)

qc = np.array([6.16128420e+04, 7.68726568e+07, 3.44609642e+03, 7.32113112e+02, 7.11335312e+02, 9.86648157e+03])
ad = np.array([1.,         1.,         1., 1., 1., 1.])

lr = 1.0
params = []
costs = []
lrs = []
N = 1000
ad_prev = np.copy(ad)
qc_prev = np.copy(qc)
cost_prev = 0.0
cost_init = False

for it in range(N):
    params.append([qc, ad])
    Delta_k = get_q_singer(dt, ad, np.ones(dim))
    Delta_k_inv = npla.inv(Delta_k)
    dQk_da = get_jac_Qk_alpha(dt, ad, qc)
    dQk_dsigma = get_jac_Qk_sigma(qc)
    for j in range(dim):
        if ad[j] < 0.05:
            dQk_da[j] = np.zeros((3 * dim, 3 * dim))
    Phi = get_tran_singer(dt, ad)
    Qk = get_q_singer(dt, ad, qc)
    Qk_inv = npla.inv(Qk)
    
    jac_ad = np.zeros(dim)
    cost = 0
    K = gamma1.shape[1]
    t = 0
    T = 1
    
    # for t in range(states_all.shape[0]):
    jac_ad_tmp = np.zeros(dim)
    s = np.zeros(dim)
    for i in range(K):
        ek = gamma2[t, i].reshape(-1, 1) - Phi @ gamma1[t, i].reshape(-1, 1)
        cost += ek.T @ Qk_inv @ ek
        dek_da = get_jac_ek_alpha(dt, ad, gamma1[t, i].reshape(-1, 1))
        for j in range(dim):
            if ad[j] >= 0.05:
                jac_ad_tmp[j] += -0.5 * np.trace(sym(ek @ ek.T) @ Qk_inv @ dQk_da[j] @ Qk_inv)
                jac_ad_tmp[j] += ek.T @ Qk_inv @ dek_da[j]
            s[j] += ek.T @ Delta_k_inv @ dQk_dsigma[j] @ ek
    for j in range(dim):
        if ad[j] >= 0.05:
            jac_ad[j] += jac_ad_tmp[j] / K + 0.5 * np.trace(Qk_inv @ dQk_da[j])
    cost += K * np.log(npla.det(Qk))
    jac_ad /= T
    cost /= T
    print("cost: {:10.4f} qc: {} jac_ad: {} ad: {}".format(cost.item(), qc, jac_ad, ad))
    qc = s / (3 * K * T)
    ad = ad - lr * jac_ad
    for j in range(ad.shape[0]):
        if ad[j] < 0.05:
            ad[j] = 0.0
            jac_ad[j] = 0.0
    costs.append(cost.item())

    if np.linalg.norm(ad - ad_prev) < 1.0e-3 and np.linalg.norm(qc - qc_prev) < 1.0e-3:
        print("Parameters converged!")
        break


    if cost.item() - cost_prev > 0 and cost_init is True:
        print("Min cost reached")
        break

    ad_prev = ad
    qc_prev = qc
    cost_prev = cost.item()
    cost_init = True

params = np.array(params)

cost: 190564.8802 qc: [6.16128420e+04 7.68726568e+07 3.44609642e+03 7.32113112e+02
 7.11335312e+02 9.86648157e+03] jac_ad: [-0.04880541 -0.04876906 -0.04895831 -0.04973766 -0.049747   -0.04877178] ad: [1. 1. 1. 1. 1. 1.]
cost: 190255.8139 qc: [6.16128420e+04 7.68726568e+07 3.44611019e+03 7.32028445e+02
 7.11252559e+02 9.86847533e+03] jac_ad: [-0.04874355 -0.04870534 -0.04890494 -0.04972484 -0.04973472 -0.04870838] ad: [1.04880541 1.04876906 1.04895831 1.04973766 1.049747   1.04877178]
cost: 189947.3816 qc: [6.16143070e+04 7.68744851e+07 3.44619331e+03 7.32045888e+02
 7.11269572e+02 9.86871004e+03] jac_ad: [-0.04868141 -0.04864135 -0.04885131 -0.04971205 -0.04972247 -0.04864453] ad: [1.09754896 1.0974744  1.09786325 1.0994625  1.09948172 1.09748016]
cost: 189639.5832 qc: [6.16158397e+04 7.68763979e+07 3.44628020e+03 7.32064170e+02
 7.11287405e+02 9.86895553e+03] jac_ad: [-0.048619   -0.04857707 -0.04879743 -0.0496993  -0.04971027 -0.0485804 ] ad: [1.14623037 1.14611575 1.14671456 1.1491

In [ ]:
qc: [6.15981907e+04 7.68543585e+07 3.44526300e+03 7.34529440e+02
 7.13699230e+02 9.86612671e+03]
ad: [0.         0.         0.         4.00139951 4.00724719 0.        ]